In [6]:
import numpy as np
import pandas as pd
import random as rm

class Die():

    def __init__(self, faces):
        """"" This is the Create_Die docstring"""
        
    # The Initializer Method: Takes an array of faces as an argument.
           
        flag = len(set(faces)) == len(faces)
        if flag == 0:
            raise ValueError("Each value must be Unique.") 
    
        #Faces
        #Asigns FACES to self, the length of the User Inputted array
        self.faces = faces
        self.n_faces = len(faces)
        self.weights = np.ones(len(faces))
    
        #Create Dataframe to save values        
        self.my_die= pd.DataFrame({
          #  'Face':range(1,self.n_faces+1),
            'Face':faces,
            'Weight':self.weights})


    def change_weight(self, f_change, weight):
        """ f_change is the name of the face - it could be a str or int"""
        # Changes_Weight Method: changes the weight of a single face provided (x) - Seems to work overall. 

        weight = float(weight)

        #Re-evaluate for string
        if f_change in self.faces:
            #self.my_die.loc[df['Face'] == f_change , "Weight" ] = weight
            self.my_die.loc[self.my_die['Face'] == f_change , "Weight" ] = weight

            #I'm finding 'H/T' and changing the weight.
            print(self.my_die)
        else:
            print("Value is not on the Die.  (i.e. User provided value is not in the array.)")
            #Raise an Error instead.

            
    def roll_die(self, n_rolls=1):
        """A method to roll the die one or more times. """
        Rolled_faces_and_weights_df = self.my_die.sample(n=n_rolls, replace = True, weights = "Weight").reset_index(drop= True)
        return list(Rolled_faces_and_weights_df['Face'])    

                                                
    def show(self, wide_or_narrow = "wide"):
        """A method to show the user the dies current set of faces and weights (since the latter can be changed). """   
        if wide_or_narrow == 'wide':
            return self.my_die
        
        elif wide_or_narrow =='narrow':
            my_die_narrow = self.my_die.stack().to_frame('Faces')
            my_die_narrow.index.rename(["Roll Number","Dice"], inplace = True)
            return my_die_narrow
        else:
            raise ValueError("Arrgument must be either \"narrow\" or \"wide\"")
    
        print(self.my_die)


In [8]:
class Game(Die): 
    
    def __init__(self,die_list):
        """Game class docstring: A game consists of rolling of one or more dice of the same kind one or more times."""      
        self.die_list=die_list       
        self.results = pd.DataFrame()
                        

    def play(self, num_rolls):
        self.my_die = pd.DataFrame()
        self.my_die.index.rename("Roll Number", inplace = True)
        
        for die in range(0, len(self.die_list)):
            self.my_die[die] = self.die_list[die].roll_die(num_rolls)
        
    
    def get_results(self, wide_or_narrow = "wide"):
       # A method to show the user the die’s current set of faces and weights (since the latter can be changed). (X)
       # Returns the dataframe created in the initializer but possibly updated by the weight changing method. (X)
    
        if wide_or_narrow == 'wide':
            return self.my_die
        
        elif wide_or_narrow =='narrow':
            my_die_narrow = self.my_die.stack().to_frame('Faces')
            my_die_narrow.index.rename(["Roll Number","Face"], inplace = True)
            return my_die_narrow
        else:
            raise ValueError("Arrgument must be either \"narrow\" or \"wide\"")
    

In [9]:

sixsided = [1,2,3,4,5,6]

#1.)
testdie1 = Die(sixsided)
testdie2 = Die(sixsided)
testdie3 = Die(sixsided)
testdie4 = Die(sixsided)
testdie5 = Die(sixsided)
testdie6 = Die(sixsided)

testdielist = [testdie1,testdie2,testdie3,testdie4]

testgame = Game(testdielist)



In [10]:
#Testing For Die Class
#1.  1.) Create Die 2.)Change Weight 3.) Roll_die  4.) Show

#Die 1 - Create Die
firstDie = Die(["1","2","3",4,5,6,7,8,9,10,11,12])

#Change Weight Method -2
firstDie.change_weight(9,"6")

#Current FW Method -3
firstDie.show("narrow")
firstDie.show("wide")


#Roll Die Method   -- Both methods work. -
firstDie.roll_die(5) 


   Face  Weight
0     1     1.0
1     2     1.0
2     3     1.0
3     4     1.0
4     5     1.0
5     6     1.0
6     7     1.0
7     8     1.0
8     9     6.0
9    10     1.0
10   11     1.0
11   12     1.0


[9, 8, 5, 9, 5]

In [11]:
#### Testing for Game class ####

#Die 1 - Create Die
first = Die(["1","2","3",4,5,6,7,8,9,10,11,12])
second = Die([1,2,3,4,5,6,7,8,9,10,11,12])
third = Die([1,2,3,4,5,6,7,8,9,10,11,12])

#Create Game
testGame=Game([first,second,third])

# Game - 1 play
testGame.play(6)

#Game show - 2
testGame.get_results("narrow")


Faces
Roll Number Face      
0           0        2
            1       10
            2        8
1           0        4
            1       11
            2       11
2           0        8
            1        9
            2        1
3           0        8
            1        6
            2        5
4           0        8
            1        5
            2        6
5           0        1
            1        4
            2        3

In [12]:
testgame.play(10)

In [13]:
testgame.get_results("wide")

,0,1,2,3
Roll Number,,,,
0,2,4,4,5
1,6,4,5,3
2,6,1,1,2
3,2,3,4,4
4,3,5,5,1
5,5,1,4,2
6,1,1,3,1
7,2,1,1,2
8,1,2,2,3


In [14]:
class Analyzer(Game): 
    """An analyzer takes the results of a single game and computes various descriptive statistical properties about it."""
    
    face_count = pd.DataFrame()
    jackpot = pd.DataFrame()
    combos = pd.DataFrame()
    game_results = pd.DataFrame()
    temp_df = pd.DataFrame()
    
    number_of_dice = 0

    
    def __init__(self, game):
        self.game_results = game.get_results()
        self.number_of_dice = len(list(self.game_results))
        
    def get_face_count(self):
        """A face counts per roll, i.e. the number of times a given face appeared in each roll. For example, if a roll of five dice has all sixes, then the counts for this roll would be 6 for the face value '6' and 0 for the other faces."""
        self.temp_df = pd.DataFrame(self.game_results.transpose().melt(var_name='rolls',value_name='faces'))
        self.face_counts = pd.crosstab(index=self.temp_df['rolls'],columns=self.temp_df['faces'])
        return self.face_counts
    
    def get_jackpot(self):
        """A jackpot count, i.e. how many times a roll resulted in all faces being the same, e.g. six ones for a six-sided die."""
        temp_df_2 = self.temp_df.groupby(['rolls','faces'], as_index = False).value_counts()
        self.jackpots = temp_df_2.loc[temp_df_2['count'] == self.number_of_dice]
        return len(self.jackpots)    
    
    def get_combo(self):
        """A combo count, i.e. how many combination types of faces were rolled and their counts."""
        self.combos = self.face_counts.groupby(list(self.face_counts), as_index = False).value_counts()
        return self.combos
    
    def get_permutation(self):
        """A permutation count, i.e. how may sequence types were rolled and their counts."""
        self.combos = self.face_counts.groupby(list(self.face_counts), as_index = False).value_counts()
        return self.combos
        

In [15]:
#A face counts per roll, i.e. the number of times a given face appeared in each roll. For example, if a roll of five dice has all sixes, then the counts for this roll would be 6 for the face value '6' and 0 for the other faces.
#A jackpot count, i.e. how many times a roll resulted in all faces being the same, e.g. six ones for a six-sided die.
#A combo count, i.e. how many combination types of faces were rolled and their counts.
#A permutation count, i.e. how may sequence types were rolled and their counts.

analysis = Analyzer(testgame)

#Method 1
analysis.get_face_count()

#Method 2
#analysis.get_combo()

#analysis.get_jackpot()

faces,1,2,3,4,5,6
rolls,,,,,,
0,0,1,0,2,1,0
1,0,0,1,1,1,1
2,2,1,0,0,0,1
3,0,1,1,2,0,0
4,1,0,1,0,2,0
5,1,1,0,1,1,0
6,3,0,1,0,0,0
7,2,2,0,0,0,0
8,1,2,1,0,0,0


In [16]:
##Scenario 1
fair_coin = Die(["H","T"])

unfair_coin = Die(["H","T"])
unfair_coin.change_weight("H",5)

#Get this to support alpha-characters.

Number_of_flips = 1000

Fair_coin_set = [fair_coin, fair_coin, fair_coin]
Fair_game = Game(fair_coin_set)
Fair_game.play(number_of_flips)

Fair_game_analyzer = Analyzer(fair_game)
Fair_game_analyzer.get_face_counts()
Fair_game_analyzer.get_combos()

Fair_relative_frequency = fair_game_analyzer.get_jackpots()/number_of_flips

Value_df = pd.DataFrame({"fair":[fair_relative_frequency], "unfair": [unfair_relative_frequency]})

#Graph
Value_df.plot.bar()
Plt.xlabel("Fairness")
Plt.ylabel("Relative Frequency")
Plt.title("Scenario 1: 2-headed coin")


  Face  Weight
0    H     5.0
1    T     1.0


NameError: name 'fair_coin_set' is not defined

In [ ]:
if __name__ == '__main__':
    unittest.main(verbosity=3)

#montycarlo.py

if  _name_ == '__main__':
    my_die = Die()
    my_die.change_weights(3,10)
    my_die.show_pie()

    my_game = Game([my_die for i in range(5)])
    my_game.play(100)

    my_analyzer = Analyzer(my_game)
    my_analyzer.find_combos()
    print(my_analyzer.combos)

In [1]:
from pathlib import Path
print(Path.cwd())


C:\Users\Ben\OneDrive\Documents\UVA 1st Semester\Programming New Github\DS5100-2022-08-O\final-project-info


In [47]:
fair_dice = Die([1,2,3,4,5,6,7])

#unfair_Die1 = Die()
#unfair_Die2 = Die()
#Does there need to be a Die parameter if one isn't passed?  like defaults to [1,2,3,4,5,6]

#unfair_die1.change_weight(6,5)
#unfair_die2.change_weight(1 ,5)

n2 = 1000
n_dice = 5

game_2_1 = Game([fair_dice for i in range(n_dice)])

game_2_1.play(n2)
game_2_1.get_results()


game_2_1.get_results('narrow').reset_index().value_counts(['Roll Number','Faces']).to_frame('n').unstack(fill_value=0)
game_2_1.get_results('narrow').reset_index()

#game_2_1.get_results('narrow').reset_index().value_counts(['Roll Number','Faces']).to_frame('n')

#X = game_2_1.get_results('narrow').reset_index().value_counts(['Roll Number','Faces']).to_frame('n')
#(X == n_dice).sum().sum()



,Roll Number,Face,Faces
0,0,0,1
1,0,1,7
2,0,2,3
3,0,3,6
4,0,4,5
...,...,...,...
4995,999,0,7
4996,999,1,4
4997,999,2,7
4998,999,3,7
